In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import xlrd
import time
from IPython.display import Audio

In [2]:
#full runtime indicator start
full_time=time.time()

In [3]:
#finish indicator
wow = '../../Data/Wow.mp3'
finished=Audio(url = wow, autoplay = True) 

In [4]:
#notebook complete indicator
notebook = '../../Data/Hugo Good Job.mp3'
complete=Audio(url = notebook, autoplay = True)

In [5]:
map = gpd.read_file('../../Data/Taxi Zones Shape(original, do not use)/taxi_zones.shp').to_crs('epsg:4326')
map = map.drop(['Shape_Area', 'Shape_Leng', 'OBJECTID'], axis=1)


complete

In [6]:
df=pd.read_csv('../../Data/Taxi Data/yellow_15/yellow_tripdata_2015-03.csv')

In [7]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

df.info()

df2=df.head(100)

df2.columns

In [8]:
start_time = time.time()

df['dropoff_geometry'] = df.apply(lambda x: Point((float(x['dropoff_longitude']), 
                                             float(x['dropoff_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 804.3608689308167 seconds ---


In [9]:
start_time = time.time()

df['pickup_geometry'] = df.apply(lambda x: Point((float(x['pickup_longitude']), 
                                             float(x['pickup_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 803.6871309280396 seconds ---


In [10]:
df.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dropoff_geometry,pickup_geometry
0,1,2015-03-06 08:02:31,2015-03-06 08:09:55,1,1.2,-73.990211,40.750969,1,N,-73.987892,...,2,7.0,0.0,0.5,0.0,0.0,0.3,7.8,POINT (-73.98789215087889 40.738037109375),POINT (-73.99021148681641 40.75096893310546)
1,1,2015-03-06 08:02:31,2015-03-06 08:15:23,1,3.2,-73.935188,40.800720,1,N,-73.952553,...,2,11.5,0.0,0.5,0.0,0.0,0.3,12.3,POINT (-73.95255279541014 40.76537322998047),POINT (-73.93518829345702 40.80072021484375)
2,1,2015-03-06 08:02:31,2015-03-06 08:12:27,1,1.1,-73.963753,40.767937,1,N,-73.956947,...,2,8.0,0.0,0.5,0.0,0.0,0.3,8.8,POINT (-73.95694732666014 40.78026962280273),POINT (-73.96375274658203 40.76793670654297)


In [11]:
df2_start=df[['passenger_count','trip_distance','fare_amount','mta_tax','tip_amount','tolls_amount','pickup_geometry']]

In [12]:
df2_end=df[['dropoff_geometry']]

In [13]:
start_time = time.time()
df2_start=gpd.GeoDataFrame(df2_start,
                     crs=map.crs,
                     geometry=df2_start['pickup_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 106.18006134033203 seconds ---


In [14]:
start_time = time.time()
df2_end=gpd.GeoDataFrame(df2_end,
                     crs=map.crs,
                     geometry=df2_end['dropoff_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 108.35509419441223 seconds ---


In [15]:
start_time = time.time()

geo_processing_start = gpd.sjoin(df2_start, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 1080.9253189563751 seconds ---


In [16]:
start_time = time.time()

geo_processing_end = gpd.sjoin(df2_end, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 1075.112858057022 seconds ---


In [17]:
geo_processing_start

,passenger_count,trip_distance,fare_amount,mta_tax,tip_amount,tolls_amount,pickup_geometry,geometry,index_right,zone,LocationID,borough
0,1,1.20,7.0,0.5,0.00,0.0,POINT (-73.99021148681641 40.75096893310546),POINT (-73.99021 40.75097),99,Garment District,100,Manhattan
12,1,0.40,3.5,0.5,0.00,0.0,POINT (-73.99166870117188 40.75164413452149),POINT (-73.99167 40.75164),99,Garment District,100,Manhattan
40,1,4.30,15.5,0.5,2.00,0.5,POINT (-73.98426818847656 40.75480651855469),POINT (-73.98427 40.75481),99,Garment District,100,Manhattan
65,1,1.19,5.5,0.5,1.36,0.0,POINT (-73.98792266845702 40.75479888916016),POINT (-73.98792 40.75480),99,Garment District,100,Manhattan
77,2,0.90,5.5,0.5,0.00,0.0,POINT (-73.98966217041014 40.75646591186523),POINT (-73.98966 40.75647),99,Garment District,100,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...
9788537,1,2.40,10.0,0.5,0.00,0.0,POINT (-73.81972503662109 40.60599899291992),POINT (-73.81973 40.60600),29,Broad Channel,30,Queens
10153791,1,3.50,12.5,0.5,0.00,2.0,POINT (-73.82012176513672 40.60371398925781),POINT (-73.82012 40.60371),29,Broad Channel,30,Queens
12154351,1,3.10,10.5,0.5,0.00,2.0,POINT (-73.82075500488281 40.59939193725586),POINT (-73.82076 40.59939),29,Broad Channel,30,Queens
12108228,2,1.60,7.5,0.5,0.00,0.0,POINT (-74.12196350097656 40.55043792724609),POINT (-74.12196 40.55044),109,Great Kills Park,110,Staten Island


In [18]:
group_start = geo_processing_start.groupby(['LocationID']).describe()

In [19]:
group_start.columns = ['_'.join(col) for col in group_start.columns.values]

group_start.to_csv('../../Data/yellow_tripdata_2014-2-start-processed.csv')

In [20]:
group_end = geo_processing_end.value_counts(['LocationID'])

In [21]:
group_end = pd.DataFrame(group_end,columns=['dropoff_count'])

group_end.to_csv('../../Data/yellow_tripdata_2014-2-end-processed.csv')

In [22]:
merge=group_start.merge(group_end, how='left', on='LocationID')

In [23]:
merge

,passenger_count_count,passenger_count_mean,passenger_count_std,passenger_count_min,passenger_count_25%,passenger_count_50%,passenger_count_75%,passenger_count_max,trip_distance_count,trip_distance_mean,...,tolls_amount_max,index_right_count,index_right_mean,index_right_std,index_right_min,index_right_25%,index_right_50%,index_right_75%,index_right_max,dropoff_count
LocationID,,,,,,,,,,,,,,,,,,,,,
1,797.0,1.351317,0.827929,0.0,1.0,1.0,1.0,9.0,797.0,2.494103,...,75.00,797.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19862
2,6.0,2.333333,2.065591,1.0,1.0,1.0,4.0,5.0,6.0,4.818333,...,0.00,6.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,9
3,42.0,1.261905,0.586828,1.0,1.0,1.0,1.0,4.0,42.0,5.408333,...,5.54,42.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,797
4,45065.0,1.685255,1.336491,0.0,1.0,1.0,2.0,6.0,45065.0,2.734256,...,60.01,45065.0,3.0,0.0,3.0,3.0,3.0,3.0,3.0,71274
5,2.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0,2.0,1.000000,...,0.00,2.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,104.0,1.355769,0.933916,1.0,1.0,1.0,1.0,6.0,104.0,3.182404,...,5.54,104.0,258.0,0.0,258.0,258.0,258.0,258.0,258.0,1687
260,9248.0,1.688365,1.404848,0.0,1.0,1.0,2.0,6.0,9248.0,3.248882,...,44.44,9248.0,259.0,0.0,259.0,259.0,259.0,259.0,259.0,18292
261,60961.0,1.699989,1.328753,0.0,1.0,1.0,2.0,6.0,60961.0,4.059131,...,80.01,60961.0,260.0,0.0,260.0,260.0,260.0,260.0,260.0,60146


In [24]:
#REMEMBER TO RENAME CSV BEFORE START
merge.to_csv('../../Data/yellow_tripdata_2015-03-processed.csv')
complete

In [25]:
print("--- %s seconds ---" % (time.time() - full_time))

--- 4027.5163650512695 seconds ---
